In [1]:
!pip install pandas
!pip install numpy

In [14]:
import pandas as pd
import os
import json
import numpy as np
from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from matplotlib import pyplot

In [3]:
liwc_features = pd.read_csv('./LIWC2015 Results (data-kushal).csv')
liwc_features = liwc_features.fillna(0)
liwc_features = liwc_features.drop(['Source (A)', 'Source (C)', 'Source (D)'], axis = 1)
liwc_features = liwc_features.rename(columns={'Source (B)': 'video_id'})

In [4]:
# Read all the data
labels = []
labels_mapping = {}
label_count = 0
df = pd.read_csv('./raw_data.csv')

for index, row in df.iterrows():
    path = os.path.join('./processed_video', 'audio_' + row['video_id'] + '_txt')
    
    if os.path.isfile(path):
        with open(path, 'r') as f:
            try:
                file_data = json.load(f)
                labels.append((row['video_likeCount'] - row['video_dislikeCount'])/row['video_viewCount'])
                labels_mapping[row['video_id']] = label_count
                label_count = label_count + 1
                
            except ValueError:
                print(path + ' failed processing')

labels = np.array(labels)
print()
print('Shape of labels: ', labels.shape)

./processed_video/audio_xCgk9nvuCxk_txt failed processing
./processed_video/audio_Q_ouhkdo-ko_txt failed processing

Shape of labels:  (3389,)


In [5]:
label_series = pd.Series(labels)
print(label_series.value_counts(bins=20).sort_index())

(-0.021500000000000002, -0.0122]       1
(-0.0122, -0.00398]                    7
(-0.00398, 0.00421]                  225
(0.00421, 0.0124]                   1094
(0.0124, 0.0206]                    1021
(0.0206, 0.0288]                     613
(0.0288, 0.037]                      264
(0.037, 0.0452]                      103
(0.0452, 0.0533]                      31
(0.0533, 0.0615]                      17
(0.0615, 0.0697]                       5
(0.0697, 0.0779]                       2
(0.0779, 0.0861]                       1
(0.0861, 0.0943]                       1
(0.0943, 0.102]                        0
(0.102, 0.111]                         1
(0.111, 0.119]                         1
(0.119, 0.127]                         0
(0.127, 0.135]                         1
(0.135, 0.143]                         1
dtype: int64


In [6]:
# Prepare data for cnn lstm by concatenating frames, normalizing labels and defining constants
labels = np.clip(labels, 0.00421, 0.037)
min_label = np.min(labels)
max_label = np.max(labels)
scaled_labels = (labels - min_label)/(max_label - min_label)

y = []
for label in scaled_labels:
    if label >= 0.0 and label < 0.2:
        y.append(0)
    elif label >= 0.2 and label < 0.4:
        y.append(1)
    elif label >= 0.4 and label < 0.6:
        y.append(2)
    elif label >= 0.6 and label < 0.8:
        y.append(3)
    elif label >= 0.8 and label <= 1.0:
        y.append(4)

y = np.array(y)

In [7]:
inputs = np.zeros((3389, 50))

for video_id, index in labels_mapping.items():
    video_features = liwc_features.loc[liwc_features.video_id == video_id, ['compare', 'i', 'Colon', 'home', 'focuspast', 'pronoun', 'ppron', 'anx', 'Dash',
 'Sixltr', 'Analytic', 'auxverb', 'friend', 'OtherP', 'informal', 'number',
 'assent', 'WC', 'we', 'WPS', 'function', 'verb', 'negemo', 'family', 'Apostro',
 'ipron', 'feel', 'prep', 'body', 'relig', 'sexual', 'insight', 'cogproc',
 'filler', 'they', 'conj', 'leisure', 'Dic', 'netspeak', 'negate', 'focuspresent',
 'QMark', 'AllPunc', 'affiliation', 'work', 'space', 'article', 'Authentic',
 'health', 'death']].to_numpy()
    video_features = np.mean(video_features, axis = 0)
    inputs[index, :] = video_features
    del video_features

/usr/local/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.7/site-packages/numpy/core/_methods.py:154: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [8]:
print('Shape of input features: (%s,%s)' % inputs.shape)
inputs[np.where(np.isnan(inputs))] = 0

Shape of input features: (3389,50)


In [9]:
indices = np.arange(inputs.shape[0])
np.random.shuffle(indices)
data = inputs[indices]
labels = y[indices]
nb_validation_samples = int(0.3 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]

x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

print('Shape of training tensor:', x_train.shape)
print('Shape of training labels:', y_train.shape)
print('Shape of testing tensor:', x_val.shape)
print('Shape of testing labels:', y_val.shape)

Shape of training tensor: (2373, 50)
Shape of training labels: (2373,)
Shape of testing tensor: (1016, 50)
Shape of testing labels: (1016,)


In [12]:
dtree_model = DecisionTreeClassifier(max_depth = 10).fit(x_train, y_train) 
y_pred = dtree_model.predict(x_val) 
y_true = y_val
print(classification_report(y_true, y_pred, labels = [0,1,2,3,4], zero_division=0))
print()
print("The final F1 micro score for the model based one arly fusion is: ", f1_score(y_true, y_pred, average = 'micro', labels=[0,1,2,3,4]))
print()
print("Detailed Confusion Matrix")
print()
print(confusion_matrix(y_true, y_pred, labels = [0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.40      0.48      0.43       332
           1       0.29      0.27      0.28       271
           2       0.26      0.31      0.28       196
           3       0.17      0.11      0.13       111
           4       0.13      0.07      0.09       106

    accuracy                           0.31      1016
   macro avg       0.25      0.25      0.24      1016
weighted avg       0.29      0.31      0.29      1016


The final F1 micro score for the model based one arly fusion is:  0.3080708661417323

Detailed Confusion Matrix

[[160  85  55  17  15]
 [109  74  59  16  13]
 [ 58  48  60  17  13]
 [ 36  32  26  12   5]
 [ 41  18  33   7   7]]


In [15]:
model = BaggingClassifier(base_estimator=SVC(C = 20), n_estimators=20, max_features=10, max_samples=50)
model.fit(x_train, y_train)

BaggingClassifier(base_estimator=SVC(C=20, break_ties=False, cache_size=200,
                                     class_weight=None, coef0=0.0,
                                     decision_function_shape='ovr', degree=3,
                                     gamma='scale', kernel='rbf', max_iter=-1,
                                     probability=False, random_state=None,
                                     shrinking=True, tol=0.001, verbose=False),
                  bootstrap=True, bootstrap_features=False, max_features=10,
                  max_samples=50, n_estimators=20, n_jobs=None, oob_score=False,
                  random_state=None, verbose=0, warm_start=False)

In [16]:
y_true = y_val
y_pred = model.predict(x_val)
print(classification_report(y_true, y_pred, labels = [0,1,2,3,4], zero_division=0))
print()
print("The final F1 micro score for the model based one arly fusion is: ", f1_score(y_true, y_pred, average = 'micro', labels=[0,1,2,3,4]))
print()
print("Detailed Confusion Matrix")
print()
print(confusion_matrix(y_true, y_pred, labels = [0, 1, 2, 3, 4]))

              precision    recall  f1-score   support

           0       0.33      0.97      0.49       332
           1       0.33      0.06      0.10       271
           2       0.00      0.00      0.00       196
           3       0.00      0.00      0.00       111
           4       0.00      0.00      0.00       106

    accuracy                           0.33      1016
   macro avg       0.13      0.21      0.12      1016
weighted avg       0.20      0.33      0.19      1016


The final F1 micro score for the model based one arly fusion is:  0.3316929133858268

Detailed Confusion Matrix

[[321  11   0   0   0]
 [255  16   0   0   0]
 [187   9   0   0   0]
 [109   2   0   0   0]
 [ 96  10   0   0   0]]
